# Parsl and RADICAL-Pilot Integration

RADICAL-Pilot (RP) is a runtime system that enables the execution of heterogeneous (funtions and executables) MPI workloads on heterogeneous (GPUs and CPUs) HPC resources. The integration of Parsl and RP (RPEX) allows RP to benefit from Parsl flexible programming model and its workflow management capabilities to build dynamic workflows. Additionally, RadicalPilotExecutor benefits Parsl by offering the heterogeneous runtime capabilities of RP to support many MPI computations more efficiently.

First and as a best practice, let's ensure RADICAL-Pilot and Parsl exist in the notebook environment.

In [ ]:
!pip show parsl && echo "==============" && ! radical-stack

Next, we locate the installed `nwchem` executable in our environment. We install it from conda-forge into the local Python environment if it is unavailable.

In [ ]:
nwchem_path = !which nwchem

if not nwchem_path:
    import sys
    !conda install --yes --prefix {sys.prefix} -c conda-forge nwchem openmpi
    nwchem_path = !which nwchem

nwchem = nwchem_path[0]

## Example: MPI NWChem Workload

The following example application shows the execution of MP2 geometry optimization followed by a CCSD(T) energy evaluation at the converged geometry. A Dunning correlation-consistent triple-zeta basis is used. The default of Cartesian basis functions must be overridden using the keyword spherical on the BASIS directive. The 1s core orbitals are frozen in both the MP2 and coupled-cluster calculations (note that these must separately specified).

First, we import the Parsl and RP Python modules in our application, alongside the RadicalPilotExecutor (RPEX) from Parsl

In [ ]:
import parsl
import radical.pilot as rp

from parsl.config import Config
from parsl.app.app import bash_app
from parsl.executors.radical import ResourceConfig
from parsl.executors.radical import RadicalPilotExecutor

`RadicalPilotExecutor` is capable of executing both functions and executables concurrently. The functions execution layer is based on the manager-worker paradigm. The managers are responsible for managing a set of workers and can execute function tasks as well. In contrast, the workers are only responsible for the function tasks execution. The manager-worker paradigm requires a set of input parameters for resource distribution, such as:
1. Number of managers and workers per node
2. Number of ranks per manager and worker.
3. Number of nodes per manager and worker.
4. Etc.

In order to specify this information, we use a configuration class `ResourceConfig` that describes these parameters and pass it to `RadicalPilotExecutor`. In the cell below, we ask `RadicalPilotExecutor` to allocate 8 cores for all tasks.

In [ ]:
rpex_cfg = ResourceConfig()

config = Config(executors=[RadicalPilotExecutor(
                           rpex_cfg=rpex_cfg,
                           label='RPEX-MPI',
                           resource='local.localhost_test',
                           runtime=30, cores=8)])

radical_executor = config.executors[0]

parsl.load(config)

Create a simple Parsl `@bash_app` to invoke the `NWChem` task. The `bash_app` requires the type of the task and the number of `ranks` on which to run. In this case, the type of the task is `MPI` as the number of `ranks` (processes) is 2, where each rank takes 1 core.

Once the `bash_app` (executable task) is invoked, the `RadicalPilotExecutor` submits the task to the runtime system and wait for them to be executed. Finally we obtain the task.stdout by specifying the `stdout` keyword when we create/invoke the `bash_app`.

In [ ]:
@bash_app
def nwchem_mp2_optimization(stdout='nwchem.stdout',
                            parsl_resource_specification={'ranks':2,
                                                          'mode': rp.TASK_EXECUTABLE}):

    import os

    input = """
    start n2

    geometry
      symmetry d2h
      n 0 0 0.542
    end

    basis spherical
      n library cc-pvtz
    end

    mp2
      freeze core
    end

    task mp2 optimize

    ccsd
      freeze core
    end

    task ccsd(t)
    """

    nwchem_input = os.path.join(os.getcwd(), 'mp2_optimization.nw')

    with open(nwchem_input,'w+') as f:
        f.writelines(input)

    return '{0} {1}'.format(nwchem, nwchem_input)

In [ ]:
# invoke the nwchem_mp2_optimization
future  = nwchem_mp2_optimization()

# wait for the results of the NWChem task.
future.result()

! echo "nwchem_mp2_optimization output:" && cat nwchem.stdout

Finally, shutdown the executor, otherwise it will always stays ready to get more tasks

In [ ]:
radical_executor.shutdown()